.. _nb_subset_selection:

## Subset Selection Problem

A genetic algorithm can be used to approach subset selection problems by defining custom operators. In general a metaheuristic algorithm might not be the ultimate goal to implement in a real-world scenario, however, it might be useful to investigate patterns or characteristics of possible good subsets. 
Let us consider a simple toy problem where we have to select numbers from a list. For every solution exactly 10 numbers have to be selected that their sum is minimized.
For subset selection problem a binary encoding can be used where 1 indicates a number is picked. In our problem formulation the list of numbers is represented by $L$ and the binary encoded variable by $x$.


\begin{align}
\begin{split}
\min f(x) & = & \sum_{k=1}^{n} L_k \cdot x_k\\[2mm]
\text{s.t.} \quad g(x)  & = & (\sum_{k=1}^{n} x_k - 10)^2\\[2mm]
\end{split}
\end{align}


As shown above, the equality constraint is handled by making sure $g(x)$ can only be zero if exactly 10 numbers are chosen.
The problem can be implemented as follows:

In [1]:
import numpy as np
from pymoo.model.problem import Problem

class SubsetProblem(Problem):
    def __init__(self,
                 L,
                 n_max
                 ):
        super().__init__(n_var=len(L), n_obj=1, n_constr=1, elementwise_evaluation=True)
        self.L = L
        self.n_max = n_max

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = np.sum(self.L[x])
        out["G"] = (self.n_max - np.sum(x)) ** 2
    
    
# create the actual problem to be solved
np.random.seed(1)
L = np.array([np.random.randint(100) for _ in range(100)])
n_max = 10
problem = SubsetProblem(L, n_max)


The customization requires to write custom operators in order to solve this problem efficiently. We recommend to consider the feasibility directly in the evolutionary operators, because otherwise most of the time infeasible solutions will be processed.
The sampling creates randomly solution where the subset constraint will always be satisfied. 
The mutation randomly removes a number and chooses another one. The crossover, first takes the common numbers of both parents and then randomly picks either from the first or from the second parent until enough numbers are picked.

In [2]:
from pymoo.model.crossover import Crossover
from pymoo.model.mutation import Mutation
from pymoo.model.sampling import Sampling


class MySampling(Sampling):

    def _do(self, problem, n_samples, **kwargs):
        X = np.full((n_samples, problem.n_var), False, dtype=np.bool)

        for k in range(n_samples):
            I = np.random.permutation(problem.n_var)[:problem.n_max]
            X[k, I] = True

        return X


class BinaryCrossover(Crossover):
    def __init__(self):
        super().__init__(2, 1)

    def _do(self, problem, X, **kwargs):
        n_parents, n_matings, n_var = X.shape

        _X = np.full((self.n_offsprings, n_matings, problem.n_var), False)

        for k in range(n_matings):
            p1, p2 = X[0, k], X[1, k]

            both_are_true = np.logical_and(p1, p2)
            _X[0, k, both_are_true] = True

            n_remaining = problem.n_max - np.sum(both_are_true)

            I = np.where(np.logical_xor(p1, p2))[0]

            S = I[np.random.permutation(len(I))][:n_remaining]
            _X[0, k, S] = True

        return _X


class MyMutation(Mutation):
    def _do(self, problem, X, **kwargs):
        for i in range(X.shape[0]):
            X[i, :] = X[i, :]
            is_false = np.where(np.logical_not(X[i, :]))[0]
            is_true = np.where(X[i, :])[0]
            X[i, np.random.choice(is_false)] = True
            X[i, np.random.choice(is_true)] = False

        return X

After having defined the operators a genetic algorithm can be initialized.

In [3]:
from pymoo.algorithms.so_genetic_algorithm import GA
from pymoo.optimize import minimize

algorithm = GA(
    pop_size=100,
    sampling=MySampling(),
    crossover=BinaryCrossover(),
    mutation=MyMutation(),
    eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               ('n_gen', 60),
               seed=1,
               verbose=True)

print("Function value: %s" % res.F[0])
print("Subset:", np.where(res.X)[0])


n_gen |  n_eval |   cv (min)   |   cv (avg)   |     favg     |     fopt    
    1 |     100 |  0.00000E+00 |  0.00000E+00 |  4.43940E+02 |          258
    2 |     200 |  0.00000E+00 |  0.00000E+00 |  3.53730E+02 |          205
    3 |     300 |  0.00000E+00 |  0.00000E+00 |  2.96710E+02 |          158
    4 |     400 |  0.00000E+00 |  0.00000E+00 |  2.46040E+02 |          140
    5 |     500 |  0.00000E+00 |  0.00000E+00 |  2.13890E+02 |          126
    6 |     600 |  0.00000E+00 |  0.00000E+00 |  1.86910E+02 |          117
    7 |     700 |  0.00000E+00 |  0.00000E+00 |  1.61650E+02 |           97
    8 |     800 |  0.00000E+00 |  0.00000E+00 |  1.42650E+02 |           97
    9 |     900 |  0.00000E+00 |  0.00000E+00 |  1.30070E+02 |           67
   10 |    1000 |  0.00000E+00 |  0.00000E+00 |  1.19540E+02 |           67
   11 |    1100 |  0.00000E+00 |  0.00000E+00 |  1.10810E+02 |           57
   12 |    1200 |  0.00000E+00 |  0.00000E+00 |  1.02180E+02 |           57
   13 |    1

Finally, we can compare the found subset with the optimum known simply through sorting:

In [4]:
opt = np.sort(np.argsort(L)[:n_max])
print("Optimal Subset:", opt)
print("Optimal Function Value: %s" % L[opt].sum())

Optimal Subset: [ 5  9 12 31 36 37 47 52 68 99]
Optimal Function Value: 36
